In [4]:
#import necessary libraries
import requests
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pprint
import numpy as np

In [5]:
url1 = 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086'
url2 = 'https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052'
asession = AsyncHTMLSession() #creates Async object

r = await asession.get(url2,timeout=20) #requests website from server and waits for response
#r.html.render(timeout=20)
site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'


In [6]:
patterns[0].contents

[<div class="_31jmH _2xk1x _1yupz _1i6F0"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at _1i6F0"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x _1yupz _1i6F0"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at _1i6F0"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>]

By cross checking with the website, I've determined that the classes that have '_1yupz' are downstrums, the classes that have '_3N-at' are downstrums, and the classes that have neither of those are beats you don't strum on

In [4]:
print(len(patterns)) #the song I'm testing with has 3 strum patterns so this list should hold three Tags
print(type(patterns)) #confirming that patterns is a list/ResultSet
print(type(patterns[0].children)) #checking the type for the children of the Tag
print(type(next(patterns[0].children))) 
print(type(next(patterns[0].children)['class']))
print(next(patterns[0].children)['class'])#Returns class string as a list


3
<class 'bs4.element.ResultSet'>
<class 'list_iterator'>
<class 'bs4.element.Tag'>
<class 'list'>
['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']


In [5]:
strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip

for pattern in patterns: #iterate through all strum patterns scraped
    currentPattern = [] #will hold current pattern
    count = 0
    for child in pattern.children: #iterate through all strums within the pattern
        className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
        
        print(className) #print class name for debugging
        
        if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
            currentPattern.append(1)
        elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
            currentPattern.append(2)
        else:   #if it contains this string, it is neither so append a 0
            currentPattern.append(0)
        
    strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    print("Integer Pattern:",currentPattern)
    print()
           

['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at', '_1i6F0']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz', '_1i6F0']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at', '_1i6F0']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x', '_3N-at']
Integer Pattern: [1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2]

['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
Integer Pattern: [1, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2]

['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x', '_1yupz']
['_31jmH', '_2xk1x']
['_31jmH', '_2xk1x', '_3N-at']
['_31jmH', '_2xk1x',

In [6]:
print(np.array(strumPatterns)) #should give 3x12 array because the song being tested has three strum patterns with 12 strums each

[[1 0 1 2 1 2 1 0 1 2 1 2]
 [1 0 0 1 0 2 1 0 0 1 0 2]
 [1 0 2 1 0 2 1 0 2 1 0 2]]


In [7]:
#Now ill figure out how to scrape the links for all the top songs
#url for top daily chords songs:
#'https://www.ultimate-guitar.com/explore?type[]=Chords'
asession = AsyncHTMLSession() #creates Async object
URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords'
r = await asession.get(URL,timeout=20) #requests website from server and waits for response

site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html


In [8]:
songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
print('Number of song links found:', len(songs)) #there should be 50 songs on the page

Number of song links found: 50


In [9]:
print(type(songs[0]['href']),'\n') 
print(songs[0],'\n') #checks that songs[0] is a Tag
print(songs[0]['href']) #checks if href will return the url as string
print('\n',songs[0].text) #checks song name

<class 'str'> 

<a class="_2KJtL _1mes3 kWOod" href="https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589">Perfect</a> 

https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589

 Perfect


In [7]:
#combine previous testing to make function that finds and returns strum patterns from given song url
import time



async def getPatterns(url): #function to parse for strum patterns at given url
    start_time = time.time()
    asession = AsyncHTMLSession() #creates Async object

    r = await asession.get(url,timeout=20) #requests website from server and waits for response
    #r.html.render(timeout=20)
    
    #print("Get Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    site = await r.html.arender(timeout=20) #renders the javascript, NOTE: Currently the speed bottleneck as the javascript is taking too long to render
    #site = r
    #print("Render Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
    body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
    patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'
    
    #print("Parse With Soup --- %s seconds ---" % (time.time() - start_time))
    
    strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip
    
    start_time = time.time()
    
    for pattern in patterns: #iterate through all strum patterns scraped
        currentPattern = [] #will hold current pattern
        count = 0
        for child in pattern.children: #iterate through all strums within the pattern
            className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
            if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
                currentPattern.append(1)
            elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
                currentPattern.append(2)
            else:   #if it contains this string, it is neither so append a 0
                currentPattern.append(0)

        strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    
    print("Parse Strum Patterns --- %s seconds ---" % (time.time() - start_time))    
    return strumPatterns #returns array of strum patterns, with each row being a strum pattern


In [8]:
#test function
test = await getPatterns('https://tabs.ultimate-guitar.com/tab/green-day/boulevard-of-broken-dreams-chords-146744')
#test = await getPatterns('https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589')
test

Parse Strum Patterns --- 1.9073486328125e-05 seconds ---


[[1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2], [1, 1, 1, 1, 1, 1, 1, 1]]

In [12]:
for song in songs:
    print(await getPatterns(song['href']),'\n')

Parse Strum Patterns --- 1.0967254638671875e-05 seconds ---
[[1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1]] 

Parse Strum Patterns --- 4.76837158203125e-06 seconds ---
[[1, 1, 1, 1, 1, 1]] 

Parse Strum Patterns --- 1.4066696166992188e-05 seconds ---
[[1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2], [1, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2], [1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2]] 

Parse Strum Patterns --- 8.344650268554688e-06 seconds ---
[[1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1]] 

Parse Strum Patterns --- 2.4080276489257812e-05 seconds ---
[[1, 0, 0, 2, 0, 2, 0, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 2, 0, 2, 0, 2, 1, 0, 1, 2, 1, 0, 1, 2]] 

Parse Strum Patterns --- 1.2636184692382812e-05 seconds ---
[[1, 0, 1, 2, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2]] 

Parse Strum Patterns --- 1.5020370483398438e-05 seconds ---
[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 

In [9]:
#input strum pattern as list with 0=skip,1=downstrum,2=upstrum, numSongs is the number of the top songs you want to check
async def searchStrumPattern(pattern,numSongs): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    count = 0 #will hold # of songs checked
    while count < numSongs and page < 10: #each page holds 50 songs so sets limits on number of pages checked
        r = await asession.get(URL+str(page),timeout=20) #requests website from server and waits for response

        site = await r.html.arender(timeout=20) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        
        for song in songs:
            patterns = await getPatterns(song['href'])
            for row in patterns:
                if pattern == row:
                    print(song['href'],'\n',song.text,'\n')
            count += 1
            print(count)
        page += 1


In [10]:
await searchStrumPattern([1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2],200) #should be strum pattern of Boulevard of Broken Dreams which is on
#[1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]



Parse Strum Patterns --- 1.5974044799804688e-05 seconds ---
1
Parse Strum Patterns --- 5.0067901611328125e-06 seconds ---
2
Parse Strum Patterns --- 1.5020370483398438e-05 seconds ---
3
Parse Strum Patterns --- 1.3828277587890625e-05 seconds ---
4
Parse Strum Patterns --- 2.4080276489257812e-05 seconds ---
5
Parse Strum Patterns --- 1.621246337890625e-05 seconds ---
6
Parse Strum Patterns --- 4.1961669921875e-05 seconds ---
7
Parse Strum Patterns --- 1.811981201171875e-05 seconds ---
8
Parse Strum Patterns --- 1.4066696166992188e-05 seconds ---
9
Parse Strum Patterns --- 2.4080276489257812e-05 seconds ---
10
Parse Strum Patterns --- 2.09808349609375e-05 seconds ---
11
Parse Strum Patterns --- 1.621246337890625e-05 seconds ---
12
Parse Strum Patterns --- 1.5735626220703125e-05 seconds ---
13
Parse Strum Patterns --- 9.5367431640625e-07 seconds ---
14
Parse Strum Patterns --- 1.0967254638671875e-05 seconds ---
15
Parse Strum Patterns --- 1.3113021850585938e-05 seconds ---
16
Parse Strum 

Parse Strum Patterns --- 7.867813110351562e-06 seconds ---
124
Parse Strum Patterns --- 1.4066696166992188e-05 seconds ---
125
Parse Strum Patterns --- 1.6927719116210938e-05 seconds ---
126
Parse Strum Patterns --- 1.3113021850585938e-05 seconds ---
127
Parse Strum Patterns --- 8.821487426757812e-06 seconds ---
128
Parse Strum Patterns --- 2.193450927734375e-05 seconds ---
129
Parse Strum Patterns --- 1.0967254638671875e-05 seconds ---
130
Parse Strum Patterns --- 1.9311904907226562e-05 seconds ---
131
Parse Strum Patterns --- 3.0279159545898438e-05 seconds ---
132
Parse Strum Patterns --- 4.0531158447265625e-06 seconds ---
133
Parse Strum Patterns --- 2.09808349609375e-05 seconds ---
134
Parse Strum Patterns --- 2.5033950805664062e-05 seconds ---
135
Parse Strum Patterns --- 2.6226043701171875e-06 seconds ---
136
Parse Strum Patterns --- 1.3113021850585938e-05 seconds ---
137
Parse Strum Patterns --- 3.695487976074219e-05 seconds ---
138
Parse Strum Patterns --- 1.811981201171875e-05